# Linear regression in python: across time dimension for every lat lon grid

In [1]:
#!zip -r TS_months.zip TS_months


In [2]:
!zip -r TS_harm_anthro.zip TS_harm_anthro

	zip warning: name not matched: TS_harm_anthro

zip error: Nothing to do! (try: zip -r TS_harm_anthro.zip . -i TS_harm_anthro)


## Load in Dependencies 

In [129]:
import xarray as xr
import numpy as np
import cftime
import pandas as pd
from numpy import *
from scipy import stats
from netCDF4 import Dataset as nc
from netCDF4 import num2date
import netCDF4
import datetime
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt #python plotting package
from matplotlib import cm
import matplotlib.colors as mcolors
from colorspacious import cspace_converter
from collections import OrderedDict
cmaps = OrderedDict()
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import nctoolkit
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cpf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely import geometry
from collections import namedtuple
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon

## Plotting Parameters 

In [130]:
# Set for each variable
variable = 'd16Op'
plt_ylabel = "d16O Precipitation (per mill)"

In [131]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
colors = ['black', 'red', 'orange', 'grey']
var_name = ['Full', 'GHG', 'OZA', 'ANTHRO']

In [132]:
def clim_plot(c,a):
    cl = ensav(c[i])
    an = ensav(a[i])
    ax = fig.add_subplot(4,2, i+1)
    plt.plot(np.arange(cl.size), cl,  label='Climatology')
    plt.plot(np.arange(an.size),an,  label='Anomalies')
    plt.rcParams['font.size'] = '14'

## Set Paths

In [133]:
## Set Paths
basedir_atm = '/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/'
file_prefix= '.cam.h0.' 
i_model = '/b.ie12.B1850C5CN.f19_g16.LME.' ## read in 2&3 iLME Ensembles 
i_model1 = '/b.ie12.B1850CN.f19_g16.' ## read in iLME ens 1
model = '/b.e11.BLMTRC5CN.f19_g16.'


In [134]:
%%time
ens = ''
file_suffix= '.185001-200512.nc'

var = 'PRECRC_H216Or'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H216Or.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H216Or.squeeze()
    dsf.append(member)     
post_H216Or = xr.concat(dsf, dim='ensemble')

var = 'PRECSL_H216OS'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H216OS.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H216OS.squeeze()
    dsf.append(member)     
post_H216OS = xr.concat(dsf, dim='ensemble')

var = 'PRECRL_H216OR'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H216OR.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H216OR.squeeze()
    dsf.append(member)     
post_H216OR = xr.concat(dsf, dim='ensemble')

var = 'PRECSC_H216Os'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H216Os.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H216Os.squeeze()
    dsf.append(member)     
post_H216Os = xr.concat(dsf, dim='ensemble')

CPU times: user 4.81 s, sys: 270 ms, total: 5.08 s
Wall time: 13.2 s


In [135]:
%%time 
post_H216O = post_H216Or + post_H216OR +post_H216Os +post_H216OS  # adll all 16O componen

CPU times: user 22.8 ms, sys: 55.9 ms, total: 78.7 ms
Wall time: 76.6 ms


In [136]:
del post_H216Or 
del post_H216OR 
del post_H216Os 
del post_H216OS 

In [137]:
%%time
ens = ''
file_suffix= '.185001-200512.nc'

var = 'PRECRC_H218Or'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H218Or.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H218Or.squeeze()
    dsf.append(member)     
post_H218Or = xr.concat(dsf, dim='ensemble')

var = 'PRECSL_H218OS'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H218OS.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H218OS.squeeze()
    dsf.append(member)     
post_H218OS = xr.concat(dsf, dim='ensemble')

var = 'PRECRL_H218OR'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H218OR.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H218OR.squeeze()
    dsf.append(member)     
post_H218OR = xr.concat(dsf, dim='ensemble')

var = 'PRECSC_H218Os'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H218Os.squeeze()
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H218Os.squeeze()
    dsf.append(member)     
post_H218Os = xr.concat(dsf, dim='ensemble')

CPU times: user 4.68 s, sys: 245 ms, total: 4.93 s
Wall time: 11.5 s


In [138]:
post_H218O = post_H218Or + post_H218OR +post_H218Os +post_H218OS  # adll all 18O componen

In [139]:
del post_H218Or
del post_H218OR 
del post_H218Os 
del post_H218OS 

In [140]:
## Read in GHG files and Calculate dD

In [141]:
%%time 
## GHG
ens = 'GHG.'  #set ensemble 
var = 'PRECRC_H218Or'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H218Or.squeeze()
    dsg.append(member)
post_H218Or_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECSL_H218OS'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H218OS.squeeze()
    dsg.append(member)
post_H218OS_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECRL_H218OR'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H218OR.squeeze()
    dsg.append(member)
post_H218OR_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECSC_H218Os'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H218Os.squeeze()
    dsg.append(member)
post_H218Os_ghg = xr.concat(dsg, dim='ensemble')

CPU times: user 3.92 s, sys: 354 ms, total: 4.27 s
Wall time: 9.53 s


In [142]:
%%time 
post_H218O_ghg = post_H218Or_ghg + post_H218OR_ghg +post_H218Os_ghg +post_H218OS_ghg # adll all 18O componen

CPU times: user 47.1 ms, sys: 43.7 ms, total: 90.8 ms
Wall time: 89.5 ms


In [143]:
del post_H218Or_ghg # rC is convective rain 
del post_H218OR_ghg # L - large scale 
del post_H218Os_ghg #SC = snow rates convective?
del post_H218OS_ghg # SL = snow rates large scale?

In [144]:
%%time 
## GHG
ens = 'GHG.'  #set ensemble 
var = 'PRECRC_H216Or'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H216Or.squeeze()
    dsg.append(member)
post_H216Or_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECSL_H216OS'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H216OS.squeeze()
    dsg.append(member)
post_H216OS_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECRL_H216OR'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H216OR.squeeze()
    dsg.append(member)
post_H216OR_ghg = xr.concat(dsg, dim='ensemble')

var = 'PRECSC_H216Os'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H216Os.squeeze()
    dsg.append(member)
post_H216Os_ghg = xr.concat(dsg, dim='ensemble')

CPU times: user 3.82 s, sys: 275 ms, total: 4.09 s
Wall time: 10.7 s


In [145]:
%%time 
post_H216O_ghg = post_H216Or_ghg + post_H216OR_ghg +post_H216Os_ghg +post_H216OS_ghg # adll all 18O componen

CPU times: user 66.2 ms, sys: 19.3 ms, total: 85.5 ms
Wall time: 84.2 ms


In [146]:
del post_H216Or_ghg # rC is convective rain 
del post_H216OR_ghg # L - large scale 
del post_H216Os_ghg #SC = snow rates convective?
del post_H216OS_ghg # SL = snow rates large scale?

In [147]:
%%time
var_ghg =  (((post_H218O_ghg/post_H216O_ghg)-1)*1000)

CPU times: user 39.2 ms, sys: 28.1 ms, total: 67.4 ms
Wall time: 66.5 ms


In [148]:
var_full =  (((post_H218O/post_H216O)-1)*1000)

In [149]:
%%time
# OZA
ens = 'O3AER.'
var = 'PRECRC_H218Or'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H218Or.squeeze()
    dso.append(member)   
post_H218Or_oza = xr.concat(dso, dim='ensemble')

var = 'PRECSL_H218OS'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H218OS.squeeze()
    dso.append(member)   
post_H218OS_oza = xr.concat(dso, dim='ensemble')

var = 'PRECRL_H218OR'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H218OR.squeeze()
    dso.append(member)   
post_H218OR_oza = xr.concat(dso, dim='ensemble')

var = 'PRECSC_H218Os'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H218Os.squeeze()
    dso.append(member)   
post_H218Os_oza = xr.concat(dso, dim='ensemble')

CPU times: user 6.14 s, sys: 334 ms, total: 6.47 s
Wall time: 14.6 s


In [150]:
%%time
post_H218O_oza = post_H218Or_oza + post_H218OR_oza +post_H218Os_oza +post_H218OS_oza # adll all 18O componen

CPU times: user 48 ms, sys: 55.5 ms, total: 103 ms
Wall time: 102 ms


In [151]:
del post_H218Or_oza 
del post_H218OR_oza 
del post_H218Os_oza  
del post_H218OS_oza  

In [152]:
%%time
# OZA
ens = 'O3AER.'
var = 'PRECRC_H216Or'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRC_H216Or.squeeze()
    dso.append(member)   
post_H216Or_oza = xr.concat(dso, dim='ensemble')

var = 'PRECSL_H216OS'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSL_H216OS.squeeze()
    dso.append(member)   
post_H216OS_oza = xr.concat(dso, dim='ensemble')

var = 'PRECRL_H216OR'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECRL_H216OR.squeeze()
    dso.append(member)   
post_H216OR_oza = xr.concat(dso, dim='ensemble')

var = 'PRECSC_H216Os'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).PRECSC_H216Os.squeeze()
    dso.append(member)   
post_H216Os_oza = xr.concat(dso, dim='ensemble')

CPU times: user 6.15 s, sys: 301 ms, total: 6.45 s
Wall time: 12.8 s


In [153]:
%%time
post_H216O_oza = post_H216Or_oza + post_H216OR_oza +post_H216Os_oza +post_H216OS_oza # adll all 16O componen

CPU times: user 40.7 ms, sys: 63.7 ms, total: 104 ms
Wall time: 103 ms


In [154]:
del post_H216Or_oza 
del post_H216OR_oza 
del post_H216Os_oza  
del post_H216OS_oza  

In [155]:
%%time
var_oza =  (((post_H218O_oza/post_H216O_oza)-1)*1000)

CPU times: user 44.6 ms, sys: 40.1 ms, total: 84.7 ms
Wall time: 84 ms


In [156]:
%%time
ens = 'LULC.'  #Lulc ensemble  
file_suffix= '.169001-200512.nc'

var = 'PRECRC_H216Or'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECRC_H216Or.squeeze()
    dsl.append(member)
post_H216Or_lulc = xr.concat(dsl, dim='ensemble')

var = 'PRECSL_H216OS'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECSL_H216OS.squeeze()
    dsl.append(member)
post_H216OS_lulc = xr.concat(dsl, dim='ensemble')

var = 'PRECRL_H216OR' 
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECRL_H216OR.squeeze()
    dsl.append(member)
post_H216OR_lulc = xr.concat(dsl, dim='ensemble')


var = 'PRECSC_H216Os'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECSC_H216Os.squeeze()
    dsl.append(member)
post_H216Os_lulc = xr.concat(dsl, dim='ensemble')


CPU times: user 4.71 s, sys: 357 ms, total: 5.06 s
Wall time: 14 s


In [157]:
%%time 
post_H216O_lulc = post_H216Or_lulc + post_H216OR_lulc +post_H216Os_lulc +post_H216OS_lulc # adll all 16O componen

CPU times: user 51.9 ms, sys: 28.2 ms, total: 80 ms
Wall time: 78 ms


In [158]:
del post_H216Or_lulc 
del post_H216OR_lulc 
del post_H216Os_lulc  
del post_H216OS_lulc  

In [159]:
%%time
ens = 'LULC.'  #Lulc ensemble  
file_suffix= '.169001-200512.nc'

var = 'PRECRC_H218Or'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECRC_H218Or.squeeze()
    dsl.append(member)
post_H218Or_lulc = xr.concat(dsl, dim='ensemble')

var = 'PRECSL_H218OS'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECSL_H218OS.squeeze()
    dsl.append(member)
post_H218OS_lulc = xr.concat(dsl, dim='ensemble')

var = 'PRECRL_H218OR' 
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECRL_H218OR.squeeze()
    dsl.append(member)
post_H218OR_lulc = xr.concat(dsl, dim='ensemble')


var = 'PRECSC_H218Os'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).PRECSC_H218Os.squeeze()
    dsl.append(member)
post_H218Os_lulc = xr.concat(dsl, dim='ensemble')


CPU times: user 4.85 s, sys: 284 ms, total: 5.13 s
Wall time: 13.4 s


In [160]:
%%time 
post_H218O_lulc = post_H218Or_lulc + post_H218OR_lulc +post_H218Os_lulc +post_H218OS_lulc # adll all 18O componen

CPU times: user 16.4 ms, sys: 63.6 ms, total: 80.1 ms
Wall time: 78.8 ms


In [161]:
del post_H218Or_lulc 
del post_H218OR_lulc 
del post_H218Os_lulc  
del post_H218OS_lulc  

In [162]:
%%time
var_lulc =  (((post_H218O_lulc/post_H216O_lulc)-1)*1000)

CPU times: user 32.6 ms, sys: 31.7 ms, total: 64.4 ms
Wall time: 63.6 ms


In [186]:
var_anthro = xr.concat([var_ghg, var_oza, var_lulc], dim = 'ensemble')

In [164]:
# dH Control 
cntl_H218Or = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218Or/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218Or.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze() #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216Or = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216Or/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216Or.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze()

cntl_H218OR = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OR/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OR.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze() #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OR = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OR/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OR.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze()

cntl_H218Os = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218Os/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218Os.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze() #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216Os = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216Os/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216Os.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze()

cntl_H218OS = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OS/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OS.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze() #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OS = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OS/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OS.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze()

cntl_H218OI = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OI/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OI.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze() #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OI = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OI/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OI.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze()


In [ ]:
#calculate Climatology

In [165]:
## Climatology Definition 
def clim(c): 
    clim_var = c.groupby('time.month').mean('time')
    anom_var = c.groupby('time.month') - clim_var
    anom_var= anom_var.compute()
    return anom_var

In [187]:
%%time
anom_var = clim(var_full)
anom_var_ghg = clim(var_ghg)
anom_var_lulc = clim(var_lulc)
anom_var_oza = clim(var_oza)
anom_var_anthro = clim(var_anthro)

CPU times: user 2.24 s, sys: 1.21 s, total: 3.45 s
Wall time: 3.47 s


In [166]:
def ensav(e):
    ensav = e.mean(dim = 'ensemble')
    return ensav

In [167]:
def weights(w):
    weight= np.cos(np.deg2rad(w.lat))
    weight.name = "weights"
    var_weighted = w.weighted(weight)
    return var_weighted

In [168]:
def pwci(c):
    east = c.sel(lat=slice(-5,5), lon=slice(210,270))
    west = c.sel(lat=slice(-5,5), lon=slice(90,150))
    east_weight = weights(east)
    west_weight = weights(west) 
    avg_east = east_weight.mean(('lat', 'lon'))
    avg_west = west_weight.mean(('lat', 'lon'))
    pwc = (avg_east- avg_west).compute()
    
    east_post = avg_east.groupby('time.month').mean('time')
    west_post = avg_west.groupby('time.month').mean('time')
    pwc_post = pwc.groupby('time.month').mean('time')
    
    return pwc_post, east_post, west_post

In [188]:
pwc_post, east_post, west_post  = pwci(var_full)
pwc_post_ghg, east_post_ghg, west_post_ghg = pwci(var_ghg)
pwc_post_oza, east_post_oza, west_post_oza  = pwci(var_oza)
pwc_post_lulc, east_post_lulc, west_post_lulc  = pwci(var_lulc)
pwc_post_anthro, east_post_anthro, west_post_anthro  = pwci(var_anthro)

## Define functions to do Harmonic Analysis

# Calculate PWC index Regaular Climatology 

In [189]:
def pwc_clim(c):
    p = c.groupby('time.month').mean('time')
    east=  p.sel(lat=slice(-5,5), lon=slice(210,270))  # 
    west = p.sel(lat=slice(-5,5), lon=slice(90,150))
    avg_east = east.mean(('lat', 'lon'))
    avg_west = west.mean(('lat', 'lon'))
    pwc = (avg_east - avg_west).compute()
    pwc= pwc.compute()
    return pwc, avg_east, avg_west

In [190]:
%%time
pwc_var, pwc_var_east,pwc_var_west  = pwc_clim(var_full)
pwc_lulc_var, pwc_lulc_var_east,pwc_lulc_var_west = pwc_clim(var_lulc)
pwc_ghg_var, pwc_ghg_var_east,pwc_ghg_var_west = pwc_clim(var_ghg)
pwc_oza_var, pwc_oza_var_east,pwc_oza_var_west = pwc_clim(var_oza)
pwc_anthro_var, pwc_anthro_var_east,pwc_anthro_var_west = pwc_clim(var_anthro)

CPU times: user 1.95 s, sys: 1.03 s, total: 2.98 s
Wall time: 2.98 s


# Make an index for the different forcings (pwc index, eastern box, western box)

In [191]:
pwc_in = [pwc_var, pwc_ghg_var,  pwc_oza_var, pwc_oza_var pwc_anthro_var]
pwc_east = [pwc_var_east,pwc_ghg_var_east, pwc_oza_var_east, pwc_anthro_var_east] 
pwc_west = [pwc_var_west,pwc_ghg_var_west, pwc_oza_var_west, pwc_anthro_var_west] 

In [192]:
## Get the lengths of lat, lon and time
nlon=len(var_full.lon)
nlat=len(var_full.lat)
ntime=len(var_full.time)
lon = var_full.lon
lat = var_full.lat
time = var_full.time
post_time = var_oza.time
ntime_post = len(var_oza.time)

# Calculate full LME harmonics PWC index

In [175]:
tlen = len(pwc_var[0])
tlen_post = len(anom_var_oza[0])

In [176]:
## Create time variables to calculate annual mean SLP
mo=12
yr=ntime//mo
post_yr = ntime_post//mo
post_year=linspace(1850,2005,156)

In [177]:
def full_harm(tseries):
    ''' calculates sum of first two harmonics to approximate annual cycle'''
    mtot = 12
    a0 = tseries.mean().item() # calculate mean of series
    t = np.arange(0, mtot, 1, dtype='float')
    omegas = []
    coef_a = []
    coef_b = [] 
    # compute coefficients
    for p in np.arange(1, 4):
        omegas.append((2.*np.pi*(p))/(mtot))
        coef_a.append((2./mtot)*(tseries*np.cos((2*np.pi*p*t)/mtot)).sum())
        coef_b.append((2./mtot)*(tseries*np.sin((2*np.pi*p*t)/mtot)).sum())
    ## first harmonic
    tmp1 = coef_a[0].item()*np.cos(omegas[0]*t) + coef_b[0].item()*np.sin(omegas[0]*t)
    hg = a0 + tmp1
    b1.append(hg)
    ## second harmonic
    tmp2 = coef_a[1].item()*np.cos(omegas[1]*t) + coef_b[1].item()*np.sin(omegas[1]*t)
    hgg = a0 + tmp2
    b2.append(hgg)
    ## first plus second harmonic
    h = a0 + tmp1 + tmp2
    b3.append(h)
    b5 = xr.DataArray(b3, dims = ['ensemble', 'month'])
    avgt = xr.concat(b5, dim= 'ensemble').compute()
    cor_con = avgt.mean(dim = 'ensemble')  
    return(b1,b2,cor_con)

# PWC POST industrial Climatology

In [179]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_var_west)):
               test, test2, avg_full = full_harm(pwc_post[i])

In [180]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_ghg)):
               test, test2, avg_ghg = full_harm(pwc_post_ghg[i])

In [181]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_lulc)):
    test, test2, avg_lulc = full_harm(pwc_post_lulc[i])

In [182]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_oza)):
               test, test2, avg_oza = full_harm(pwc_post_oza[i])

In [183]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_anthro)):
               test, test2, avg_anthro = full_harm(pwc_post_anthro[i])

In [184]:
harm_full= [avg_full, avg_ghg, avg_oza, avg_anthro]

In [185]:
colors = 

SyntaxError: invalid syntax (1798344879.py, line 1)

In [ ]:
def harm_plot(c):
    #ax = fig.add_subplot(4,2, i+1)
    plt.plot(x, c[i],  color = colors[i], label= var_name[i])
    plt.xticks(np.arange(12), months, rotation=25) 
    plt.legend(bbox_to_anchor=(1, 0.5), loc='center left')
    plt.ylabel(plt_ylabel)
    plt.rcParams['font.size'] = '23'

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Climatology PWC Index (E-W)', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "pwc_harm_post.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza
del avg_anthro

# East POST industrial Climatology

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post)):
               test, test2, avg_full = full_harm(east_post[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_ghg)):
               test, test2, avg_ghg = full_harm(east_post_ghg[i])

In [ ]:
# b1=[]
# b2 =[]
# b3=[]
# for i in range(len(east_post_lulc)):
#                test, test2, avg_lulc = full_harm(east_post_lulc[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_oza)):
               test, test2, avg_oza = full_harm(east_post_oza[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_anthro)):
               test, test2, avg_anthro = full_harm(east_post_anthro[i])

In [ ]:
harm_full= [avg_full,avg_ghg,avg_oza, avg_anthro]

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Climatology Eastern Pacific ', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "east_harm_post.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza
del avg_anthro

# West POST industrial Climatology

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post)):
               test, test2, avg_full = full_harm(west_post[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_ghg)):
               test, test2, avg_ghg = full_harm(west_post_ghg[i])

In [ ]:
# b1=[]
# b2 =[]
# b3=[]
# for i in range(len(west_post_lulc)):
#                test, test2, avg_lulc = full_harm(west_post_lulc[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_oza)):
               test, test2, avg_oza = full_harm(west_post_oza[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_anthro)):
               test, test2, avg_anthro = full_harm(west_post_anthro[i])

In [ ]:
harm_full= [avg_full,avg_ghg,  avg_oza, avg_anthro]

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Climatology Western Pacific ', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "west_harm_post.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza
del avg_anthro

# Calculate PWC index Anomalgies calcualted relative to climatolofy

In [ ]:
pwc_post, east_post, west_post  = pwci(anom_var)
pwc_post_ghg, east_post_ghg, west_post_ghg = pwci(anom_var_ghg)
pwc_post_oza, east_post_oza, west_post_oza  = pwci(anom_var_oza)
#pwc_post_lulc, east_post_lulc, west_post_lulc  = pwci(anom_var_lulc)
pwc_post_anthro, east_post_anthro, west_post_anthro = pwci(anom_var_anthro)

# PWC POST industrial Anomaly

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post)):
               test, test2, avg_full = full_harm(pwc_post[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_ghg)):
               test, test2, avg_ghg = full_harm(pwc_post_ghg[i])

In [ ]:
# b1=[]
# b2 =[]
# b3=[]
# for i in range(len(pwc_post_lulc)):
#                test, test2, avg_lulc = full_harm(pwc_post_lulc[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_oza)):
               test, test2, avg_oza = full_harm(pwc_post_oza[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(pwc_post_anthro)):
               test, test2, avg_anthro = full_harm(pwc_post_anthro[i])

In [ ]:
harm_full= [avg_full, avg_ghg, avg_oza, avg_anthro]

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Anomaly PWC Index (E-W)', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "pwc_harm_post_anom.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza
del avg_anthro

# East POST industrial Anomaly

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post)):
               test, test2, avg_full = full_harm(east_post[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_ghg)):
               test, test2, avg_ghg = full_harm(east_post_ghg[i])

In [ ]:
# b1=[]
# b2 =[]
# b3=[]
# for i in range(len(east_post_lulc)):
#                test, test2, avg_lulc = full_harm(east_post_lulc[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_oza)):
               test, test2, avg_oza = full_harm(east_post_oza[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(east_post_anthro)):
               test, test2, avg_anthro= full_harm(east_post_anthro[i])

In [ ]:
harm_full= [avg_full,avg_ghg, avg_oza, avg_anthro]

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Anomaly Eastern Pacific ', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "east_harm_post_anom.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza
del avg_anthro

# West POST industrial Anomaly

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post)):
               test, test2, avg_full = full_harm(west_post[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_ghg)):
               test, test2, avg_ghg = full_harm(west_post_ghg[i])

In [ ]:
# b1=[]
# b2 =[]
# b3=[]
# for i in range(len(west_post_lulc)):
#                test, test2, avg_lulc = full_harm(west_post_lulc[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_oza)):
               test, test2, avg_oza = full_harm(west_post_oza[i])

In [ ]:
b1=[]
b2 =[]
b3=[]
for i in range(len(west_post_anthro)):
               test, test2, avg_anthro = full_harm(west_post_anthro[i])

In [ ]:
harm_full= [avg_full, avg_ghg, avg_oza, avg_anthro]

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
x = np.linspace(0,11,12)
for i in range(len(harm_full)):
    harm_plot(harm_full)
plt.suptitle(variable + ' Post Industrial Anomaly Western Pacific ', fontsize=40)
plt.subplots_adjust(hspace=0.5, wspace=2)
plt.tight_layout()
plt.savefig(variable + "west_harm_post_anom.jpg")

In [ ]:
del harm_full
del avg_full
del avg_ghg
#del avg_lulc
del avg_oza

In [ ]:
from itertools import product
from cftime import DatetimeNoLeap

dates = [DatetimeNoLeap(year, month, 1) for year, month in product(range(1850, 2006), range(1, 13))]

da = xr.DataArray(np.arange(1872), coords=[dates], dims=['time'], name='time')

In [ ]:
var_full = var.assign_coords({'time':(da.time)})
var_ghg = var_ghg.assign_coords({'time':(da.time)})
var_lulc = var_lulc.assign_coords({'time':(da.time)})
var_oza = var_oza.assign_coords({'time':(da.time)})

In [ ]:
var_anthro = xr.concat([var_ghg, var_oza, var_lulc], dim = 'ensemble')

In [ ]:
var_full = var_full *1000*24*3600
var_ghg = var_ghg *1000*24*3600
var_lulc = var_lulc*1000*24*3600
var_oza = var_oza*1000*24*3600

In [ ]:
## Climatology Definition 
def clim(c): 
    clim_var = c.groupby('time.month').mean('time')
    anom_var = c.groupby('time.month') - clim_var
    anom_var= anom_var.compute()
    return anom_var

In [ ]:
def ensav(e):
    ensav = e.mean(dim = 'ensemble')
    return ensav

In [ ]:
def weights(w):
    weight= np.cos(np.deg2rad(w.lat))
    weight.name = "weights"
    var_weighted = w.weighted(weight)
    return var_weighted

In [ ]:
def pwci(c):
    east = c.sel(lat=slice(-5,5), lon=slice(210,270))
    west = c.sel(lat=slice(-5,5), lon=slice(90,150))
    east_weight = weights(east)
    west_weight = weights(west) 
    avg_east = east_weight.mean(('lat', 'lon'))
    avg_west = west_weight.mean(('lat', 'lon'))
    pwc = (avg_east- avg_west).compute()
    
    east_post = avg_east.groupby('time.month').mean('time')
    west_post = avg_west.groupby('time.month').mean('time')
    pwc_post = pwc.groupby('time.month').mean('time')
    
    return pwc_post, east_post, west_post

In [ ]:
%%time
anom_var = clim(var_full)
anom_var_ghg = clim(var_ghg)
#anom_var_lulc = clim(var_lulc)
anom_var_oza = clim(var_oza)
anom_var_anthro = clim(var_anthro)

In [ ]:
pwc_post, east_post, west_post  = pwci(var_full)
pwc_post_ghg, east_post_ghg, west_post_ghg = pwci(var_ghg)
pwc_post_oza, east_post_oza, west_post_oza  = pwci(var_oza)
#pwc_post_lulc, east_post_lulc, west_post_lulc  = pwci(var_lulc)
pwc_post_anthro, east_post_anthro, west_post_anthro  = pwci(var_anthro)

## Define functions to do Harmonic Analysis